# Producing Visuals 


## Icosahedron
- Create a graph corresponding the the icosahedron sampling level of the data.  (*g = SphereIcosahedron(level=5)*)
- Generate icosahedron longitude and latitude: *icolong, icolat = np.rad2deg(g.lon), np.rad2deg(g.lat)*


## Equiangular
- Determine the longitude and latitude from the data. Dimension 0 is longitude (height) and dimension 1 is latitude (width).
- *lon_ = np.arange(x.size(0))/x.size(0)*360*
- *lat_ = np.arange(x.size(1))/x.size(1)*180-90*
- *lon, lat = np.meshgrid(lon_, lat_)*

The Following examples were performed for icosahedron data of the Climate data at "http://island.me.berkeley.edu/ugscnn/data/climate_sphere_l5.zip"


# Requirements
To run the code below please install the following.
- Cartopy:
```bash
conda install -c conda-forge cartopy
```
- Imageio
```bash
conda install -c conda-forge imageio
```
- Matplotlib
```bash
conda install -c conda-forge matplotlib
```

In [ ]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import imageio

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from torchvision import transforms
from pygsp.graphs.nngraphs.sphereicosahedron import SphereIcosahedron
from deepsphere.data.datasets.dataset import ARTCDataset
from deepsphere.data.transforms.transforms import Normalize, Permute, ToTensor
from deepsphere.utils.stats_extractor import stats_extractor

In [ ]:
def visualize_2d(x, longitude, latitude, export_path=None):
    """Visualize the data on a 2D map

    Args:
        x (numpy.array): numpy array with data the size of the longitude and latitude
        longitude (numpy.array): longitude coordinates
        latitude (numpy.array): latitude coordinates
        export_path (string): path and name for saving
    """

    fig = plt.figure(figsize=(20, 10))
    ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())

    ax.set_global()
    ax.coastlines()

    plt.scatter(longitude, latitude, s=20, c=x, cmap=plt.get_cmap("RdYlBu_r"), alpha=1)
    if export_path:
        plt.savefig(export_path)
        plt.clf()
        plt.cla()
        plt.close()

    else:
        plt.show()

In [ ]:
def visualize_3d(x, longitude, latitude, export_dir, save_format='png'):
    """Visualize the data on a 3D globe

    Args:
        x (numpy.array): numpy array with data the size of the longitude and latitude
        longitude (numpy.array): longitude coordinates
        latitude (numpy.array): latitude coordinates
        export_dir (string): path where to save images to make gif and the gif itself
        save_format (string): the format in which to save the images to make the gif. Default=png
    """

    for i in range(0, 330, 10):
        fig = plt.figure(figsize=(10, 10))
        ax = fig.add_subplot(1, 1, 1, projection=ccrs.Orthographic(i, 0))
        ax.set_global()
        ax.coastlines(linewidth=2)

        plt.scatter(longitude, latitude, s=80, c=x, cmap=plt.get_cmap("RdYlBu_r"), alpha=1, transform=ccrs.PlateCarree())  # marker='3',
        idx = str(i)
        try:
            path = export_dir + "/globes/globe_" + idx
            plt.savefig(path + "." + save_format)
            plt.clf()
            plt.cla()
            plt.close()
        except ValueError:
            raise ValueError('The export directory does not exist or you have not prepared a "globes" directory to store the images composing the gif. Please prepare a "globes" folder in your directory for the 3D vizualisation.')

    images = []
    for frame in range(0, 330, 10):
        images.append(imageio.imread(export_dir + "/globes/globe_" + str(frame) + "." + save_format))
        imageio.mimsave(export_dir + "movie_globe.gif", images, duration=0.75)

In [ ]:
g = SphereIcosahedron(level=5)
icolong, icolat = np.rad2deg(g.lon), np.rad2deg(g.lat)

In [ ]:
path_to_data = "../../../../../data/climate/data_5_all"
data = ARTCDataset(path=path_to_data, download=False)

In [ ]:
visualize_2d(data[10][1].transpose(1,0), icolong, icolat)

In [ ]:
visualize_3d(data[10][1].transpose(1,0), icolong, icolat, 'Example_3D_Icosahedron')